# **Laboratorio 11: LLM y Agentes Autónomos 🤖**

MDS7202: Laboratorio de Programación Científica para Ciencia de Datos

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### **Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Carolina Nuñez
- Nombre de alumno 2: Alonso Uribe

### **Link de repositorio de GitHub:** [Repositorio💻](https://github.com/carinunez/Labs_MDS)

## **Temas a tratar**

- Reinforcement Learning
- Large Language Models

## **Reglas:**

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

### **Objetivos principales del laboratorio**

- Resolución de problemas secuenciales usando Reinforcement Learning
- Habilitar un Chatbot para entregar respuestas útiles usando Large Language Models.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

## **1. Reinforcement Learning (2.0 puntos)**

En esta sección van a usar métodos de RL para resolver dos problemas interesantes: `Blackjack` y `LunarLander`.

In [ ]:
!pip install -qqq gymnasium stable_baselines3
!pip install -qqq swig
!pip install -qqq gymnasium[box2d]

### **1.1 Blackjack (1.0 puntos)**

<p align="center">
  <img src="https://www.recreoviral.com/wp-content/uploads/2016/08/s3.amazonaws.com-Math.gif"
" width="400">
</p>

La idea de esta subsección es que puedan implementar métodos de RL y así generar una estrategia para jugar el clásico juego Blackjack y de paso puedan ~~hacerse millonarios~~ aprender a resolver problemas mediante RL.

Comencemos primero preparando el ambiente. El siguiente bloque de código transforma las observaciones del ambiente a `np.array`:


In [ ]:
import gymnasium as gym
from gymnasium.spaces import MultiDiscrete
import numpy as np

class FlattenObservation(gym.ObservationWrapper):
    def __init__(self, env):
        super(FlattenObservation, self).__init__(env)
        self.observation_space = MultiDiscrete(np.array([32, 11, 2]))

    def observation(self, observation):
        return np.array(observation).flatten()

# Create and wrap the environment
env = gym.make("Blackjack-v1")
env = FlattenObservation(env)

#### **1.1.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [Blackjack](https://gymnasium.farama.org/environments/toy_text/blackjack/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas.

`escriba su respuesta acá`

#### **1.1.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 5000 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política? ¿Cómo podría interpretar las recompensas obtenidas?

#### **1.1.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `Blackjack`.

#### **1.1.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.1.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

#### **1.1.5 Estudio de acciones (0.2 puntos)**

Genere una función que reciba un estado y retorne la accion del agente. Luego, use esta función para entregar la acción escogida frente a los siguientes escenarios:

- Suma de cartas del agente es 6, dealer muestra un 7, agente no tiene tiene un as
- Suma de cartas del agente es 19, dealer muestra un 3, agente tiene tiene un as

¿Son coherentes sus acciones con las reglas del juego?

Hint: ¿A que clase de python pertenecen los estados? Pruebe a usar el método `.reset` para saberlo.

### **1.2 LunarLander**

<p align="center">
  <img src="https://i.redd.it/097t6tk29zf51.jpg"
" width="400">
</p>

Similar a la sección 2.1, en esta sección usted se encargará de implementar una gente de RL que pueda resolver el ambiente `LunarLander`.

Comencemos preparando el ambiente:


In [ ]:
import gymnasium as gym
env = gym.make("LunarLander-v2", render_mode = "rgb_array", continuous = True) # notar el parámetro continuous = True

Noten que se especifica el parámetro `continuous = True`. ¿Que implicancias tiene esto sobre el ambiente?

Además, se le facilita la función `export_gif` para el ejercicio 2.2.4:

In [ ]:
import imageio
import numpy as np

def export_gif(model, n = 5):
  '''
  función que exporta a gif el comportamiento del agente en n episodios
  '''
  images = []
  for episode in range(n):
    obs = model.env.reset()
    img = model.env.render()
    done = False
    while not done:
      images.append(img)
      action, _ = model.predict(obs)
      obs, reward, done, info = model.env.step(action)
      img = model.env.render(mode="rgb_array")

  imageio.mimsave("agent_performance.gif", [np.array(img) for i, img in enumerate(images) if i%2 == 0], fps=29)

#### **1.2.1 Descripción de MDP (0.2 puntos)**

Entregue una breve descripción sobre el ambiente [LunarLander](https://gymnasium.farama.org/environments/box2d/lunar_lander/) y su formulación en MDP, distinguiendo de forma clara y concisa los estados, acciones y recompensas. ¿Como se distinguen las acciones de este ambiente en comparación a `Blackjack`?

Nota: recuerde que se especificó el parámetro `continuous = True`

`escriba su respuesta acá`

#### **1.2.2 Generando un Baseline (0.2 puntos)**

Simule un escenario en donde se escojan acciones aleatorias. Repita esta simulación 10 veces y reporte el promedio y desviación de las recompensas. ¿Cómo calificaría el performance de esta política?

#### **1.2.3 Entrenamiento de modelo (0.2 puntos)**

A partir del siguiente [enlace](https://stable-baselines3.readthedocs.io/en/master/guide/algos.html), escoja un modelo de `stable_baselines3` y entrenelo para resolver el ambiente `LunarLander` **usando 10000 timesteps de entrenamiento**.

#### **1.2.4 Evaluación de modelo (0.2 puntos)**

Repita el ejercicio 1.2.2 pero utilizando el modelo entrenado. ¿Cómo es el performance de su agente? ¿Es mejor o peor que el escenario baseline?

#### **1.2.5 Optimización de modelo (0.2 puntos)**

Repita los ejercicios 1.2.3 y 1.2.4 hasta obtener un nivel de recompensas promedio mayor a 50. Para esto, puede cambiar manualmente parámetros como:
- `total_timesteps`
- `learning_rate`
- `batch_size`

Una vez optimizado el modelo, use la función `export_gif` para estudiar el comportamiento de su agente en la resolución del ambiente y comente sobre sus resultados.

Adjunte el gif generado en su entrega (mejor aún si además adjuntan el gif en el markdown).

## **2. Large Language Models (4.0 puntos)**

En esta sección se enfocarán en habilitar un Chatbot que nos permita responder preguntas útiles a través de LLMs.

### **2.0 Configuración Inicial**

<p align="center">
  <img src="https://media1.tenor.com/m/uqAs9atZH58AAAAd/config-config-issue.gif"
" width="400">
</p>

Como siempre, cargamos todas nuestras API KEY al entorno:

In [1]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

if "TAVILY_API_KEY" not in os.environ:
    os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter your Tavily API key: ")

Enter your Google AI API key: ··········
Enter your Tavily API key: ··········


### **2.1 Retrieval Augmented Generation (1.5 puntos)**

<p align="center">
  <img src="https://y.yarn.co/218aaa02-c47e-4ec9-b1c9-07792a06a88f_text.gif"
" width="400">
</p>

El objetivo de esta subsección es que habiliten un chatbot que pueda responder preguntas usando información contenida en documentos PDF a través de **Retrieval Augmented Generation.**

#### **2.1.1 Reunir Documentos (0 puntos)**

Reuna documentos PDF sobre los que hacer preguntas siguiendo las siguientes instrucciones:
  - 2 documentos .pdf como mínimo.
  - 50 páginas de contenido como mínimo entre todos los documentos.
  - Ideas para documentos: Documentos relacionados a temas académicos, laborales o de ocio. Aprovechen este ejercicio para construir algo útil y/o relevante para ustedes!
  - Deben ocupar documentos reales, no pueden utilizar los mismos de la clase.
  - Deben registrar sus documentos en la siguiente [planilla](https://docs.google.com/spreadsheets/d/1Hy1w_dOiG2UCHJ8muyxhdKPZEPrrL7BNHm6E90imIIM/edit?usp=sharing). **NO PUEDEN USAR LOS MISMOS DOCUMENTOS QUE OTRO GRUPO**
  - **Recuerden adjuntar los documentos en su entrega**.

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%pip install --upgrade --quiet PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00


In [4]:
%pip install --upgrade --quiet  langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 2.5 MB/s eta 0:00:00


In [5]:
%pip install --upgrade --quiet faiss-cpu langchain_community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.3/409.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.7 MB/s eta 0:00:00


In [6]:
import PyPDF2

doc_paths = ['/content/gdrive/MyDrive/pdf_LLM/506.pdf',
            #  '/content/gdrive/MyDrive/pdf_LLM/Encyclopedia of Foods. A Guide to Healthy Nutrition ( PDFDrive.com ).pdf',
             '/content/gdrive/MyDrive/pdf_LLM/La-guia-peq-cambios-castella.pdf',
             '/content/gdrive/MyDrive/pdf_LLM/Manual_basico_N_clinica_y_Dietetica_Valencia_2012.pdf',
             '/content/gdrive/MyDrive/pdf_LLM/cb2395en.pdf',
             '/content/gdrive/MyDrive/pdf_LLM/food-for-thought-mental-health-nutrition-briefing-march-2017.pdf',
             '/content/gdrive/MyDrive/pdf_LLM/nutrients-14-00750.pdf'] # rellenar con los path a sus documentos

assert len(doc_paths) >= 2, "Deben adjuntar un mínimo de 2 documentos"

total_paginas = sum(len(PyPDF2.PdfReader(open(doc, "rb")).pages) for doc in doc_paths)
assert total_paginas >= 50, f"Páginas insuficientes: {total_paginas}"

#### **2.1.2 Vectorizar Documentos (0.2 puntos)**

Vectorice los documentos y almacene sus representaciones de manera acorde.

In [7]:
from langchain_community.document_loaders import PyPDFLoader

docs = PyPDFLoader(doc_paths[-1]).load()

for doc in doc_paths[:-1]:
  new_doc = PyPDFLoader(doc).load()
  docs.extend(new_doc)


In [30]:
# Splits docs in chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs) # dividir documentos en chunks

In [31]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore

In [32]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

In [33]:
# question = "que frutas recomiendas para una persona deshidratada" # pregunta
# relevant_documents = retriever.invoke(question) # top k documentos relevantes a la pregunta
# relevant_documents

In [34]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### **2.1.3 Habilitar RAG (0.3 puntos)**

Habilite la solución RAG a través de una *chain* y guárdela en una variable.

In [35]:
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

retriever_chain = retriever | format_docs # chain
print(retriever_chain.invoke("lista de verduras con mayor contenido en fibra"))

MANUAL BÁSICO DE NUTRICIÓN CLÍNICA Y DIETÉTICA
76
tatas produce una respuesta glucémica del 70% respecto a una cantidad equivalente de pan blanco. La utili-
dad de este índice pierde parte de su valor debido a que las comidas incluyen mezclas de diversos alimentos.
2.2 Fibra. Es un HC complejo, no absorbible, que se encuentra principalmente en verduras, hortalizas, le-
gumbres, cereales y frutas. Se recomienda una ingesta de 20-35 g por día (10-15 g de fibra cada 1000 kcal/

CAPÍTULO 2. DIETÉTICA Y DIETOTERAPIA
47
GRUPO DE LAS VERDURAS Y HORTALIZAS
A pesar de que las verduras y las hortalizas forman una amplia gama de alimentos, pueden ser reunidas en 
un mismo grupo, ya que los nutrientes que aportan son similares. Son alimentos en general de bajo conteni-
do energético, ya que casi un 80% de su composición es agua. Aportan vitaminas A, C, ácido fólico, betaca-

rotenos y otros carotenos (licopeno, alfacaroteno), además de flavonoides, compuestos fenólicos y clorofila 
y una important

In [36]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash", # modelo de lenguaje
    temperature=0.2, # probabilidad de "respuestas creativas"
    max_tokens=None, # sin tope de tokens
    timeout=None, # sin timeout
    max_retries=2, # número máximo de intentos
)

# Entrego contexto para las respuestas
rag_template = '''
Eres un asistente experto en nutrición y salud. Haces recomendaciones sobre alimentos
a consumir según el contexto que se te entrega.
Tu único rol es contestar preguntas del usuario a partir de información relevante que te sea proporcionada.
Responde siempre de la forma más completa posible y usando toda la información entregada.
Responde sólo lo que te pregunten a partir de la información relevante, NUNCA inventes una respuesta.

Información relevante: {context}
Pregunta: {question}
Respuesta útil:
'''

rag_prompt = PromptTemplate.from_template(rag_template)

# Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:


from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

question = "que frutas recomiendas para la deshidratación?"
response = rag_chain.invoke(question)
print(response)


Las frutas y hortalizas son alimentos ricos en agua, por lo que son una buena opción para combatir la deshidratación. 



In [37]:
# question = "entregame 5 verduras con alto contenido en fibra y su cantidad, si dispones de ella'"
# response = rag_chain.invoke(question)
# print(response)

In [38]:
question = "que alimentos recomeindas para reducir la obesidad"
response = rag_chain.invoke(question)
print(response)

Para reducir la obesidad, se recomienda reducir tanto el aporte calórico global como la ingesta de grasas. Se recomienda consumir:

* **Pescado:** Dos o más raciones de pescado a la semana (con excepción de los filetes de pescado frito comerciales) ya que proporcionan ácidos grasos poliinsaturados n-3.
* **Conservas de pescado en aceite de oliva:**  
* **Conejo, pollo o pavo sin piel asado, frito en aceite de oliva o al horno:** 
* **Ternera, cerdo magro o jamón serrano, suprimiendo la grasa visible:**
* **Huevos:** 2 a la semana.
* **Nueces naturales:** diariamente. 
* **Fruta:** (naranjas, fresas, kiwi, etc.) fresca a diario.
* **Verdura:** (tomates, zanahorias, lechuga), fresca a diario.

Se deben restringir:

* **Sal y alimentos salados:**
* **Azúcar y alimentos y bebidas azucaradas:**
* **Carne roja y procesada:**
* **Alimentos ultraprocesados:** 



In [39]:
print(rag_chain.invoke('recomendaciones de meriendas dulces'))

Para una merienda dulce, puedes optar por:

* **Fruta entera:** Naranja, melocotón, melón, limón, mandarina, sandía, manzana ácida.
* **Combinaciones de fruta:** Naranja y vainilla, melocotón, limón y tomillo, pepino y mejorana, mandarina y albahaca, naranja, clavo y canela en rama, sandía y romero, hinojo y naranja o limón.
* **Fruta desecada:** Pasas, ciruelas, orejones, dátiles. 
* **Infusión:** Puedes preparar una infusión de hierbas aromáticas como tomillo, mejorana, albahaca, romero. 

Recuerda que es importante evitar el azúcar añadido y priorizar la fruta entera. 



#### **2.1.4 Verificación de respuestas (0.5 puntos)**

Genere un listado de 3 tuplas ("pregunta", "respuesta correcta") y analice la respuesta de su solución para cada una. ¿Su solución RAG entrega las respuestas que esperaba?

Ejemplo de tupla:
- Pregunta: ¿Quién es el presidente de Chile?
- Respuesta correcta: El presidente de Chile es Gabriel Boric

In [41]:
referencia = [
    ('que alimentos debe tener la dieta de una persona con diabetes?',
     #respuesta
    'Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.'),
    ('alimentos recomendados para la deshidratación',
     "Beba líquidos. A veces, los líquidos muy fríos resultan más fáciles de beber.\
     Recuerde que los alimentos contienen líquidos. Procure comer frutas, verduras,\
     sopas, gelatinas, paletas de helado y otros alimentos hidratados"),
    ('que alimentos recomeindas para reducir la obesidad',
     'nn')]

for i, ref in enumerate(referencia):
  print('Pregunta: ', ref[0])
  print('Respuesta de ref: ', ref[1])
  print('Respuesta RAG chain: ', rag_chain.invoke(ref[0]))

Pregunta:  que alimentos debe tener la dieta de una persona con diabetes?
Respuesta de ref:  Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.
Respuesta RAG chain:  La dieta de una persona con diabetes debe incluir alimentos como pan, cereales, pastas, legumbres, arroz y verduras. Estos alimentos contribuyen al aporte de fibra, especialmente la fibra soluble, que ha demostrado tener efectos beneficiosos sobre el perfil glucémico. Los glúcidos sencillos serán aportados mediante la ingesta de frutas y leche. Los horarios de las comidas en pacientes insulinodependiente se deben ajustar en función de la pauta de insulina. 

Pregunta:  alimentos recomendados para la deshidratación
Respuesta de ref:  Beba líquidos. A veces, los líquidos muy fríos resultan más fáciles de beber.     Recuerde que los alimentos contienen líquidos. Procure comer frutas, verduras,     sopas, gelatinas, paletas de helado y otros alimentos h

#### **2.1.5 Sensibilidad de Hiperparámetros (0.5 puntos)**

Extienda el análisis del punto 2.1.4 analizando cómo cambian las respuestas entregadas cambiando los siguientes hiperparámetros:
- `Tamaño del chunk`. (*¿Cómo repercute que los chunks sean mas grandes o chicos?*)
- `La cantidad de chunks recuperados`. (*¿Qué pasa si se devuelven muchos/pocos chunks?*)
- `El tipo de búsqueda`. (*¿Cómo afecta el tipo de búsqueda a las respuestas de mi RAG?*)

In [42]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

retriever_chain = retriever | format_docs # chain

In [43]:
rag_prompt = PromptTemplate.from_template(rag_template)

# Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

for i, ref in enumerate(referencia):
  print('Pregunta: ', ref[0])
  print('Respuesta de ref: ', ref[1])
  print('Respuesta RAG chain: ', rag_chain.invoke(ref[0]))

Pregunta:  que alimentos debe tener la dieta de una persona con diabetes?
Respuesta de ref:  Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.
Respuesta RAG chain:  La dieta de una persona con diabetes debe incluir alimentos frescos, poco procesados o sin procesar y principalmente vegetales, evitando los alimentos de baja calidad nutricional y las bebidas alcohólicas. 

Pregunta:  alimentos recomendados para la deshidratación
Respuesta de ref:  Beba líquidos. A veces, los líquidos muy fríos resultan más fáciles de beber.     Recuerde que los alimentos contienen líquidos. Procure comer frutas, verduras,     sopas, gelatinas, paletas de helado y otros alimentos hidratados
Respuesta RAG chain:  La mejor bebida para la deshidratación es el agua, tanto durante las comidas como entre ellas. 
Si algún día quieres tomar una bebida distinta, puedes elegir un agua con gas, un zumo de fruta sin azúcares añadidos, una infu

In [44]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

retriever_chain = retriever | format_docs # chain

In [45]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 3}, # n° documentos a recuperar
                                     )

retriever_chain = retriever | format_docs # chain
rag_prompt = PromptTemplate.from_template(rag_template)

# Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

for i, ref in enumerate(referencia):
  print('Pregunta: ', ref[0])
  print('Respuesta de ref: ', ref[1])
  print('Respuesta RAG chain: ', rag_chain.invoke(ref[0]))

Pregunta:  que alimentos debe tener la dieta de una persona con diabetes?
Respuesta de ref:  Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.
Respuesta RAG chain:  La dieta de una persona con diabetes debe ser individualizada, pero en general debe incluir:

* **Carbohidratos complejos:** Estos se encuentran en alimentos como el pan, cereales, pastas, legumbres, arroz y verduras. Estos alimentos también aportan fibra, que es beneficiosa para el control del azúcar en sangre.
* **Frutas:** Las frutas aportan glúcidos sencillos y fibra.
* **Proteínas:** Las proteínas deben estar presentes en la dieta, pero no en exceso.
* **Grasas:** Las grasas deben ser saludables, como las que se encuentran en el aceite de oliva, los frutos secos y los pescados.

Es importante tener en cuenta que la cantidad de cada tipo de alimento debe ser ajustada según las necesidades individuales de cada persona. 

Pregunta:  alimentos reco

Al aumentar el tamaño del chunk, se tiene más contexto por lo que tiene sentido que las respuestas entregadas la segunda prueba tengan mayor extension.

In [46]:
# Cambio de numero de chunks a recuperar
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
vectorstore
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 6}, # n° documentos a recuperar
                                     )

retriever_chain = retriever | format_docs # chain
rag_prompt = PromptTemplate.from_template(rag_template)

# Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

for i, ref in enumerate(referencia):
  print('Pregunta: ', ref[0])
  print('Respuesta de ref: ', ref[1])
  print('Respuesta RAG chain: ', rag_chain.invoke(ref[0]))

Pregunta:  que alimentos debe tener la dieta de una persona con diabetes?
Respuesta de ref:  Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.
Respuesta RAG chain:  La dieta de una persona con diabetes debe ajustarse de forma individualizada, teniendo en cuenta la valoración del estado nutricional, su estilo de vida y los objetivos terapéuticos marcados. 

**En general, la dieta debe incluir:**

* **Alimentos ricos en fibra:** pan, cereales, pastas, legumbres, arroz y verduras. Estos alimentos contribuyen al aporte de fibra, especialmente la fibra soluble, que ha demostrado tener efectos beneficiosos sobre el perfil glucémico.
* **Frutas:** aportan glúcidos sencillos.
* **Leche:** aporta glúcidos sencillos.
* **Alimentos con control de carbohidratos:** se deben ajustar en función de la pauta de insulina utilizada, adaptada a cada paciente, y al perfil glucémico.

**Se deben evitar o limitar:**

* **Alimentos fl

In [49]:
# Cambio de numero de chunks a recuperar
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento
retriever = vectorstore.as_retriever(search_type="similarity", # método de búsqueda
                                     search_kwargs={"k": 2}, # n° documentos a recuperar
                                     )

retriever_chain = retriever | format_docs # chain
rag_prompt = PromptTemplate.from_template(rag_template)

# Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

for i, ref in enumerate(referencia):
  print('Pregunta: ', ref[0])
  print('Respuesta de ref: ', ref[1])
  print('Respuesta RAG chain: ', rag_chain.invoke(ref[0]))

Pregunta:  que alimentos debe tener la dieta de una persona con diabetes?
Respuesta de ref:  Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.
Respuesta RAG chain:  La dieta de una persona con diabetes debe incluir alimentos como pan, cereales, pastas, legumbres, arroz y verduras. Estos alimentos contribuyen al aporte de fibra, especialmente la fibra soluble, que ha demostrado tener efectos beneficiosos sobre el perfil glucémico. Los glúcidos sencillos serán aportados mediante la ingesta de frutas y leche. Los horarios de las comidas en pacientes insulinodependiente se deben ajustar en función de la pauta. 

Pregunta:  alimentos recomendados para la deshidratación
Respuesta de ref:  Beba líquidos. A veces, los líquidos muy fríos resultan más fáciles de beber.     Recuerde que los alimentos contienen líquidos. Procure comer frutas, verduras,     sopas, gelatinas, paletas de helado y otros alimentos hidratados
Re

# cambio en num archivos por recuperar

In [51]:
# Cambio de numero de chunks a recuperar
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50) # inicializamos splitter
splits = text_splitter.split_documents(docs)
embedding = GoogleGenerativeAIEmbeddings(model="models/embedding-001") # inicializamos los embeddings
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding) # vectorizacion y almacenamiento

# Maximum Marginal Relevance Search
retriever = vectorstore.as_retriever(search_type='mmr', search_kwargs={"k": 2})

retriever_chain = retriever | format_docs # chain
rag_prompt = PromptTemplate.from_template(rag_template)

# Finalmente, condensamos todo en una chain para recuperar información relevante y responder al mismo tiempo:

rag_chain = (
    {
        "context": retriever_chain, # context lo obtendremos del retriever_chain
        "question": RunnablePassthrough(), # question pasará directo hacia el prompt
    }
    | rag_prompt # prompt con las variables question y context
    | llm # llm recibe el prompt y responde
    | StrOutputParser() # recuperamos sólo la respuesta
)

for i, ref in enumerate(referencia):
  print('Pregunta: ', ref[0])
  print('Respuesta de ref: ', ref[1])
  print('Respuesta RAG chain: ', rag_chain.invoke(ref[0]))

Pregunta:  que alimentos debe tener la dieta de una persona con diabetes?
Respuesta de ref:  Frutas.Verduras.Granos integrales.Legumbres, como frijoles y guisantes. Productos lácteos bajos en grasa, como leche y queso.
Respuesta RAG chain:  La dieta de una persona con diabetes debe incluir alimentos como pan, cereales, pastas, legumbres, arroz y verduras. Estos alimentos contribuyen al aporte de fibra, especialmente la fibra soluble, que ha demostrado tener efectos beneficiosos sobre el perfil glucémico. Las frutas y la leche aportan glúcidos sencillos. 

Pregunta:  alimentos recomendados para la deshidratación
Respuesta de ref:  Beba líquidos. A veces, los líquidos muy fríos resultan más fáciles de beber.     Recuerde que los alimentos contienen líquidos. Procure comer frutas, verduras,     sopas, gelatinas, paletas de helado y otros alimentos hidratados
Respuesta RAG chain:  Los alimentos recomendados para la deshidratación son:

* **Pescado blanco:** Hervido, a la plancha o pasado p

### **2.2 Agentes (1.0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/rcqnN2aJCSEAAAAd/secret-agent-man.gif"
" width="400">
</p>

Similar a la sección anterior, en esta sección se busca habilitar **Agentes** para obtener información a través de tools y así responder la pregunta del usuario.

#### **2.2.1 Tool de Tavily (0.2 puntos)**

Generar una *tool* que pueda hacer consultas al motor de búsqueda **Tavily**.

In [59]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tavily = TavilySearchResults(max_results = 1) # inicializamos tool
tools = [search_tavily] # guardamos las tools en una lista

#### **2.2.2 Tool de Wikipedia (0.2 puntos)**

Generar una *tool* que pueda hacer consultas a **Wikipedia**.

*Hint: Le puede ser de ayuda el siguiente [link](https://python.langchain.com/v0.1/docs/modules/tools/).*

In [60]:
%pip install wikipedia

In [61]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=100)
search_wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

tools += [search_wiki]

In [66]:
[search_tavily] + [search_wiki]

[TavilySearchResults(max_results=1, api_wrapper=TavilySearchAPIWrapper(tavily_api_key=SecretStr('**********'))),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/usr/local/lib/python3.10/dist-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=100))]

#### **2.2.3 Crear Agente (0.3 puntos)**

Crear un agente que pueda responder preguntas preguntas usando las *tools* antes generadas. Asegúrese que su agente responda en español. Por último, guarde el agente en una variable.

#### **2.2.4 Verificación de respuestas (0.3 puntos)**

Pruebe el funcionamiento de su agente y asegúrese que el agente esté ocupando correctamente las tools disponibles. ¿En qué casos el agente debería ocupar la tool de Tavily? ¿En qué casos debería ocupar la tool de Wikipedia?

### **2.3 Multi Agente (1.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/r7QMJLxU4BoAAAAd/this-is-getting-out-of-hand-star-wars.gif"
" width="450">
</p>

El objetivo de esta subsección es encapsular las funcionalidades creadas en una solución multiagente con un **supervisor**.


#### **2.3.1 Generando Tools (0.5 puntos)**

Transforme la solución RAG de la sección 2.1 y el agente de la sección 2.2 a *tools* (una tool por cada uno).

#### **2.3.2 Agente Supervisor (0.5 puntos)**

Habilite un agente que tenga acceso a las tools del punto anterior y pueda responder preguntas relacionadas. Almacene este agente en una variable llamada supervisor.

#### **2.3.3 Verificación de respuestas (0.25 puntos)**

Pruebe el funcionamiento de su agente repitiendo las preguntas realizadas en las secciones 2.1.4 y 2.2.4 y comente sus resultados. ¿Cómo varían las respuestas bajo este enfoque?

#### **2.3.4 Análisis (0.25 puntos)**

¿Qué diferencias tiene este enfoque con la solución *Router* vista en clases? Nombre al menos una ventaja y desventaja.

`escriba su respuesta acá`

### **2.4 Memoria (Bonus +0.5 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/Gs95aiElrscAAAAd/memory-unlocked-ratatouille-critic.gif"
" width="400">
</p>

Una de las principales falencias de las soluciones que hemos visto hasta ahora es que nuestro chat no responde las interacciones anteriores, por ejemplo:

- Pregunta 1: "Hola! mi nombre es Sebastián"
  - Respuesta esperada: "Hola Sebastián! ..."
- Pregunta 2: "Cual es mi nombre?"
  - Respuesta actual: "Lo siento pero no conozco tu nombre :("
  - **Respuesta esperada: "Tu nombre es Sebastián"**

Para solucionar esto, se les solicita agregar un componente de **memoria** a la solución entregada en el punto 2.3.

**Nota: El Bonus es válido <u>sólo para la sección 2 de Large Language Models.</u>**

### **2.5 Despliegue (0 puntos)**

<p align="center">
  <img src="https://media1.tenor.com/m/IytHqOp52EsAAAAd/you-get-a-deploy-deploy.gif"
" width="400">
</p>

Una vez tengan los puntos anteriores finalizados, toca la etapa de dar a conocer lo que hicimos! Para eso, vamos a desplegar nuestro modelo a través de `gradio`, una librería especializada en el levantamiento rápido de demos basadas en ML.

Primero instalamos la librería:

In [47]:
%pip install --upgrade --quiet gradio

Luego sólo deben ejecutar el siguiente código e interactuar con la interfaz a través del notebook o del link generado:

In [48]:
import gradio as gr
import time

def agent_response(message, history):
  '''
  Función para gradio, recibe mensaje e historial, devuelte la respuesta del chatbot.
  '''
  # get chatbot response
  response = ... # rellenar con la respuesta de su chat

  # assert
  assert type(response) == str, "output de route_question debe ser string"

  # "streaming" response
  for i in range(len(response)):
    time.sleep(0.015)
    yield response[: i+1]

gr.ChatInterface(
    agent_response,
    type="messages",
    title="Chatbot MDS7202", # Pueden cambiar esto si lo desean
    description="Hola! Soy un chatbot muy útil :)", # también la descripción
    theme="soft",
    ).launch(
        share=True, # pueden compartir el link a sus amig@s para que interactuen con su chat!
        debug = False,
        )

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8519dba04d0882d0e1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
